In [41]:
from z3 import *
from itertools import product
from functools import reduce

s = Solver()


In [42]:

δ = Function('δ', IntSort(), IntSort(), RealSort())
R = Function('R', IntSort(), RealSort())
B = Function('B', IntSort(), RealSort())

# --- aircraft etc as before ---
p1, a, p2, b, p3 = Ints('p1 a p2 b p3')
ac = [p1, a, p2, b, p3]

x, y = Ints('x y')
s.add(ForAll([x], R(x) >= 0))
s.add(ForAll([x], B(x) >= 0))
s.add(ForAll([x], R(x) >= B(x)))
s.add(ForAll([x, y], δ(x, y) >= 0))

# δ-identical a/b
for x_ in ac:
    s.add(δ(a, x_) == δ(b, x_))
    s.add(δ(x_, a) == δ(x_, b))



In [43]:
α = Real("α");
s.add(
    α == 2
)

# Define the Objective Function

In [43]:
# Define ω weightings
ω1, ω2, ω3, ω4 = Reals('ω1 ω2 ω3 ω4')

# Constraints on what ω can be.
s.add(ω1 >= 0, ω2 >= 0, ω3 >= 0, ω4 >= 0)
s.add(ω1 <= ω3, ω2 <= ω4)

def C(t, lc):
    return If(t <= lc,
              RealVal(0),
              If(t <= lc + 300,
                 ω1 * (t - lc) + ω2,
                 ω3 * (t - lc) + ω4))

In [44]:
# zmax and T computation
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs):
    m = xs[0]
    for x in xs[1:]:
        m = If(x >= m, x, m)
    return m

def compute_T(seq):
    T = {seq[0]: R(seq[0])}
    for i in range(1, len(seq)):
        preds = seq[:i]
        T[seq[i]] = zmax(R(seq[i]), zmax_list([T[x] + δ(x, seq[i]) for x in preds]))
    return T


In [45]:

S1, S2 = [p1, a, p2, b, p3], [p1, b, p2, a, p3]
T1, T2 = compute_T(S1), compute_T(S2)

W1 = Real('W1')
s.add(W1 > 0)
def delay_cost(T):
    return Sum(([W1 * (T[x] - B(x) ** α) for x in ac]))

D1, D2 = delay_cost(T1), delay_cost(T2)
s.add(R(a) <= R(b), B(a) <= B(b))
s.add(D1 > D2)


In [46]:

if s.check() == z3.sat:
    model = s.model()
    print("\nTheorem 4 holds SAT (counterexample found).\nSatisfying Model:")
    m = s.model()
    for d in m.decls():
        print(f" - {d.name()} = {m[d]}")
elif s.check() == z3.unsat:
    print("\nUNSAT — Theorem 4 holds: a before b cannot worsen hard-window compliance if s' is feasible.")
else:
    print("\nUNKOWN - Theorem 4 exceeds SMT decidable logic.")




UNSAT — Theorem 4 holds: a before b cannot worsen hard-window compliance if s' is feasible.


In [47]:
# visualise
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.table import Table



def z3_to_float(val):
    if is_rational_value(val):
        return val.numerator_as_long() / val.denominator_as_long()
    if is_algebraic_value(val):
        s = val.approx(20)
        if s.endswith("?"): s = s[:-1]
        return float(s)
    s = str(val)
    if s.endswith("?"): s = s[:-1]
    if "/" in s:
        num, den = s.split("/", 1)
        return float(num) / float(den)
    return float(s)

if s.check() == sat:
    print("SAT, Generating Counterexample...")
    m = s.model()
    data = []
    for ac in aircraft:
        r   = z3_to_float(m.evaluate(R[ac], model_completion=True))
        lt  = z3_to_float(m.evaluate(LT[ac], model_completion=True))
        t   = z3_to_float(m.evaluate(T[ac], model_completion=True))
        tp  = z3_to_float(m.evaluate(T_prime[ac], model_completion=True))
        ok_s  = t  <= lt
        ok_sp = tp <= lt
        data.append({
            "Aircraft": str(ac),
            "R": r,
            "LT": lt,
            "T(S)": t,
            "T(S′)": tp,
            "OK(S)": ok_s,
            "OK(S′)": ok_sp,
        })

    df = pd.DataFrame(data)

    delta_matrix = np.array([
        [z3_to_float(m.evaluate(δ[a, b], model_completion=True)) for b in aircraft]
        for a in aircraft
    ])
    labels = [str(a) for a in aircraft]

    fig, (ax_heat, ax_table) = plt.subplots(1, 2, figsize=(10, 4),
                                            gridspec_kw={'width_ratios': [1, 1.4]})
    plt.subplots_adjust(wspace=0.4)

    im = ax_heat.imshow(delta_matrix, cmap="viridis", interpolation="nearest")

    ax_heat.set_title("Separation Matrix δ(x, y)")
    ax_heat.set_xticks(np.arange(len(labels)))
    ax_heat.set_yticks(np.arange(len(labels)))
    ax_heat.set_xticklabels(labels)
    ax_heat.set_yticklabels(labels)
    plt.setp(ax_heat.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Draw numeric values on top of heatmap
    for i in range(len(labels)):
        for j in range(len(labels)):
            val = delta_matrix[i, j]
            text_color = "white" if val < np.max(delta_matrix) / 2 else "black"
            ax_heat.text(j, i, f"{val:.2f}",
                         ha="center", va="center", color=text_color, fontsize=9, fontweight='bold')

    fig.colorbar(im, ax=ax_heat, fraction=0.046, pad=0.04)

    ax_table.axis("off")
    tbl = Table(ax_table, bbox=[0, 0, 1, 1])

    n_rows, n_cols = len(df), 5
    col_labels = ["Aircraft", "R", "LT", "T(S)", "T(S′)"]
    col_widths = [0.2, 0.15, 0.15, 0.25, 0.25]
    row_height = 1.0 / (n_rows + 1)

    for j, label in enumerate(col_labels):
        cell = tbl.add_cell(0, j, width=col_widths[j], height=row_height,
                            text=label, loc='center', facecolor='#40466e')
        cell.get_text().set_color('w')
        cell.get_text().set_weight('bold')

    for i, row in enumerate(df.itertuples(), start=1):
        vals = [row.Aircraft, f"{row.R:.3f}", f"{row.LT:.3f}",
                f"{row._4:.3f}", f"{row._5:.3f}"]
        for j, val in enumerate(vals):
            facecolor = 'white'
            if col_labels[j] == "T(S)":
                facecolor = '#c8e6c9' if row._6 else '#ffcdd2'   # green/red
            elif col_labels[j] == "T(S′)":
                facecolor = '#c8e6c9' if row._7 else '#ffcdd2'
            cell = tbl.add_cell(i, j, width=col_widths[j], height=row_height,
                                text=val, loc='center', facecolor=facecolor)
            cell.get_text().set_color('black')

    ax_table.add_table(tbl)
    ax_table.set_title("Timing and Window Compliance", fontsize=12, pad=12)

    plt.show()